# Skillbox — Мастер-класс: Разведочный анализ данных + Подготовка отчётов — Kiva.org
***

# Новый раздел

# 4. Домашнее задание
***

https://www.kaggle.com/kiva/data-science-for-good-kiva-crowdfunding/download

In [1]:
import numpy as np
import pandas as pd
import math
#import missingno as msno
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df_kiva_loans = pd.read_csv("kiva_loans.csv")
df_mpi_reg_loc = pd.read_csv("kiva_mpi_region_locations.csv")
df_loan_theme_ids = pd.read_csv("loan_theme_ids.csv")
df_loan_themes_by_reg = pd.read_csv("loan_themes_by_region.csv")

In [3]:
print('Страны, в которых преобладают группы людей: ')

group_country = df_kiva_loans[df_kiva_loans.borrower_genders.notna()]\
    .groupby('country')\
    .agg(is_group_more = ('borrower_genders',\
        lambda x: sum([-1 if i == 'male' or i == 'female' else 1 for i in list(x.values)]) > 0))
group_country[group_country.is_group_more].drop(columns=['is_group_more'])

Страны, в которых преобладают группы людей: 


""
country
Benin
Brazil
Burkina Faso
Burundi
Cambodia
Chile
Congo
Dominican Republic
Ghana


## 4.1. Самостоятельная работа
***

- А в каких странах преобладают мужчины? Группа людей? (п. 2.2.2.)
- Изучите несколько займов с очень большими суммами. Что можно сказать о назначении этих заявок? В каких странах они оформлены? Каков гендерный состав заявителей? (п. 2.4.2.)
- Попробуйте найти связь между странами (регионами), полом заёмщика и суммой заявки. Какие выводы можно сделать из этих данных? (п. 2.4.3.)

- А в каких странах преобладают мужчины? Группа людей? (п. 2.2.2.)

In [4]:
print('Страны, в которых преобладают мужчины: ')

group_country = df_kiva_loans[df_kiva_loans.borrower_genders.notna()]\
    .groupby('country')\
    .agg(is_male_more = ('borrower_genders',\
        lambda x: sum([i.count('male') - 2 * i.count('female') for i in list(x.values)]) > 0))
group_country[group_country.is_male_more].drop(columns=['is_male_more'])

Страны, в которых преобладают мужчины: 


""
country
Azerbaijan
Belize
China
Egypt
Kosovo
Lao People's Democratic Republic
Mauritania
Mongolia
Nigeria


In [5]:
print('Страны, в которых преобладают группы людей: ')

group_country = df_kiva_loans[df_kiva_loans.borrower_genders.notna()]\
    .groupby('country')\
    .agg(is_group_more = ('borrower_genders',\
        lambda x: sum([-1 if i == 'male' or i == 'female' else 1 for i in list(x.values)]) > 0))
group_country[group_country.is_group_more].drop(columns=['is_group_more'])

Страны, в которых преобладают группы людей: 


""
country
Benin
Brazil
Burkina Faso
Burundi
Cambodia
Chile
Congo
Dominican Republic
Ghana


#  
- Изучите несколько займов с очень большими суммами. Что можно сказать о назначении этих заявок? В каких странах они оформлены? Каков гендерный состав заявителей? (п. 2.4.2.)

In [6]:
df_kiva_loans.sort_values('funded_amount', ascending = False).head(20)

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
70499,722883,100000.0,100000.0,Agriculture,Agriculture,create more than 300 jobs for women and farmer...,HT,Haiti,Les Cayes,USD,315.0,2014-06-10 19:25:02+00:00,2014-09-08 07:00:00+00:00,2014-06-19 20:21:04+00:00,75.0,2986,"user_favorite, user_favorite, user_favorite, u...",female,irregular,2014-06-10
492809,1152957,50000.0,50000.0,Agriculture,Agriculture,"to expand weather, farming information and fin...",GH,Ghana,Accra,USD,490.0,2016-09-19 13:08:02+00:00,2016-11-30 08:00:00+00:00,2016-10-17 18:36:56+00:00,14.0,1481,"#Technology, #Technology, #Sustainable Ag, use...",male,irregular,2016-09-19
604502,1266423,50000.0,50000.0,Agriculture,Agriculture,to add value and jobs to the local economy by ...,BJ,Benin,Parakou,USD,478.0,2017-03-24 21:53:03+00:00,2017-02-22 08:00:00+00:00,2017-04-18 06:42:43+00:00,12.0,1519,"user_favorite, user_favorite, user_favorite, u...",male,irregular,2017-03-24
490191,1150277,50000.0,50000.0,Health,Health,To purchase raw materials in order to produce ...,GH,Ghana,Accra,USD,489.0,2016-09-14 13:03:24+00:00,2016-11-30 08:00:00+00:00,2016-09-19 20:06:54+00:00,14.0,1569,"#Health and Sanitation, #Biz Durable Asset, #E...",male,irregular,2016-09-14
126839,777718,50000.0,50000.0,Agriculture,Agriculture,to buy and plant resin producing pine trees. T...,MX,Mexico,Cherán,USD,376.0,2014-10-01 20:46:15+00:00,2014-08-31 07:00:00+00:00,2014-12-07 17:02:10+00:00,144.0,586,"user_favorite, user_favorite, #Biz Durable Ass...","male, male, male, male, male, male, male, female",irregular,2014-10-01
538248,1198658,50000.0,50000.0,Agriculture,Agriculture,"to enable 5,000 additional small-holder farmer...",KE,Kenya,Nanyuki,USD,505.0,2016-12-05 15:00:24+00:00,2017-01-31 08:00:00+00:00,2016-12-19 23:47:20+00:00,14.0,1589,"user_favorite, user_favorite, #Eco-friendly, #...",male,monthly,2016-12-05
408295,1055043,50000.0,50000.0,Clothing,Clothing,to set up a garment social business that will ...,AL,Albania,Cerrik,USD,315.0,2016-04-15 16:09:28+00:00,2016-05-16 07:00:00+00:00,2016-04-21 17:25:45+00:00,97.0,1626,"user_favorite, user_favorite, user_favorite, u...",male,monthly,2016-04-15
523659,1183916,50000.0,50000.0,Electronics Sales,Retail,to train & equip 200 rural merchants in Mozamb...,MZ,Mozambique,Maputo,USD,492.0,2016-11-12 02:45:29+00:00,2016-12-31 08:00:00+00:00,2016-12-18 14:10:28+00:00,14.0,1410,"#Biz Durable Asset, #Technology, #Job Creator,...",female,irregular,2016-11-12
565733,1226382,50000.0,50000.0,Agriculture,Agriculture,to pay 600 farming families 100% above market ...,EC,Ecuador,Quito,USD,514.0,2017-01-25 00:25:00+00:00,2017-03-31 07:00:00+00:00,2017-02-21 10:49:58+00:00,14.0,1689,"user_favorite, user_favorite, user_favorite, u...",female,irregular,2017-01-25
621860,1283951,50000.0,50000.0,Renewable Energy Products,Retail,to distribute 400 pay-as-you-go solar home sys...,KE,Kenya,Nairobi,USD,526.0,2017-04-21 22:46:44+00:00,2017-05-31 07:00:00+00:00,2017-05-01 15:26:03+00:00,14.0,1170,"#Eco-friendly, #Technology, user_favorite, use...",male,irregular,2017-04-21


In [9]:
df_kiva_loans.sort_values('funded_amount', ascending = False).head(100).plot.bar(x='funded_amount', y='borrower_genders', rot=0)

TypeError: no numeric data to plot

## 4.2. Домашняя работа
***

Расширьте наш отчёт исследованием следующих вопросов:

- Зависимость пола заёмщика от региона планеты, сектора экономики.
- Влияние такого атрибута займа, как **Количество кредиторов** (lender_count), на суммы и сроки займов.
- Влияние показателя **MPI** (многомерный индекс бедности) в разных странах на суммы займов и сроки погашения.
- Зависимости таких метрик, как «Сумма займа», «Срок займа», «Время финансирования заявки», «Ежемесячный платёж», в разрезе **макрорегионов**.